<a href="https://colab.research.google.com/github/DiaaZiada/Nazra/blob/master/NazRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/DiaaZiada/Nazra.git
!mv Nazra/data data/
!mv Nazra/Model Model
!rm -r Nazra

Cloning into 'Nazra'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 248 (delta 47), reused 84 (delta 34), pack-reused 149
Receiving objects: 100% (248/248), 2.67 MiB | 9.12 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [0]:
from time import time
import torch
from torch import nn

In [0]:
!nvidia-smi

Tue May  7 20:19:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8    18W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


0
1
Tesla T4


In [0]:
torch.cuda.is_available()

True

In [0]:
from os import listdir
from os.path import isfile, join
import time

# https://discuss.pytorch.org/t/simple-way-to-inverse-transform-normalization/4821/2
class UnNormalize(object):
    def __init__(self, mean, std, img=False):
        self.mean = mean
        self.std = std
        self.img = img

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
            if self.img:
                t.mul_(255)
        return tensor

def cudafy(object, use_cuda):
    if use_cuda:
        return object.cuda()
    else:
        return object

def display_est_time_loop(tot_toc, curr_ix, tot_iter, prefix=''):
    if curr_ix == tot_iter:
        neat_time = time.strftime('%H:%M:%S', time.gmtime(tot_toc))
        print("\r" + prefix + 'Total elapsed time (HH:MM:SS): ' + neat_time, end='')
        print('')
    else:
        avg_toc = tot_toc / curr_ix
        estimated_time_hours = (avg_toc * (tot_iter - curr_ix))
        neat_time = time.strftime('%H:%M:%S', time.gmtime(estimated_time_hours))
        perc = str(round(curr_ix*100/tot_iter))
        print('\r' + prefix + 'Estimated time (HH:MM:SS): ' + neat_time + ' ' + perc + '%', end='')
    return tot_toc

def _print_layer_output_shape(layer_name, output_shape):
    print("Layer " + layer_name + " output shape: " + str(output_shape))

def swap_cols(arr, frm, to):
    arr[:,[frm, to]] = arr[:,[to, frm]]

def myprint(msg, filepath=None):
    print(msg)
    if not filepath is None:
        with open(filepath, 'a') as f:
            f.write(msg + '\n')

def list_files_in_dir(dir):
    return [f for f in listdir(dir) if isfile(join(dir, f))]

In [0]:
import torch.nn as nn
import torch

def NetBlockConvBatchRelu(in_channels, filters, kernel_size, stride,
                          padding=0):
    return nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=filters,
                      kernel_size=kernel_size, stride=stride,
                      padding=padding),
            nn.BatchNorm2d(num_features=filters),
            nn.ReLU()
        )

def NetBlocksSequenceConvBatchRelu(num_blocks, in_channels, out_channels,
                                   kernel_sizes, strides, paddings=0):
    if paddings == 0:
        paddings = [0] * num_blocks
    conv_sequence = []
    conv_sequence.append(NetBlockConvBatchRelu(in_channels, out_channels[0],
                                               kernel_sizes[0], strides[0],
                                               padding=paddings[0]))
    for i in range(num_blocks - 1):
        conv_sequence.append(NetBlockConvBatchRelu(out_channels[i], out_channels[i+1],
                                                   kernel_sizes[i+1], strides[i+1],
                                                   paddings[i+1]))
    conv_sequence = nn.Sequential(*conv_sequence)
    return conv_sequence

def NetBlocksDeconvBatchRelu(in_channels, out_channels, kernel_size, stride,
                          padding=0):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size,
                           stride=stride, padding=padding),
        nn.BatchNorm2d(num_features=out_channels),
        nn.ReLU()
    )

def NetBlocksSequenceDeconvBatchRelu(num_blocks, in_channels, out_channels,
                                   kernel_sizes, strides, paddings=0):
    if paddings == 0:
        paddings = [0] * num_blocks
    deconv_sequence = []
    deconv_sequence.append(NetBlocksDeconvBatchRelu(in_channels, out_channels[0],
                                               kernel_sizes[0], strides[0],
                                               padding=paddings[0]))
    for i in range(num_blocks-1):
        deconv_sequence.append(NetBlockConvBatchRelu(out_channels[i], out_channels[i+1],
                                                   kernel_sizes[i+1], strides[i+1],
                                                   paddings[i+1]))
    deconv_sequence = nn.Sequential(*deconv_sequence)
    return deconv_sequence

class NetBlocksFlatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class NetBlockSoftmaxLogProbability2D(torch.nn.Module):
    def __init__(self):
        super(NetBlockSoftmaxLogProbability2D, self).__init__()

    def forward(self, x):
        orig_shape = x.data.shape
        seq_x = []
        for channel_ix in range(orig_shape[1]):
            softmax_ = F.softmax(x[:, channel_ix, :, :].contiguous()
                                 .view((orig_shape[0], orig_shape[2] * orig_shape[3])), dim=1)\
                .view((orig_shape[0], orig_shape[2], orig_shape[3]))
            seq_x.append(softmax_.log())
        x = torch.stack(seq_x, dim=1)
        return x


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

def _print_layer_output_shape(layer_name, output_shape):
    print("Layer " + layer_name + " output shape: " + str(output_shape))

def HALNetResConvSequence(stride, filters1, filters2,
                          padding1=1, padding2=0, padding3=0,
                          first_in_channels=0):
    if first_in_channels == 0:
        first_in_channels = filters1
    return nn.Sequential(
        # added padding = 1 to make shapes fit when joining
        # with left module
        NetBlockConvBatchRelu(kernel_size=1, stride=stride, filters=filters1,
                              in_channels=first_in_channels, padding=padding1),
        NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=filters1,
                              in_channels=filters1, padding=padding2),
        NetBlockConvBatchRelu(kernel_size=1, stride=1, filters=filters2,
                              in_channels=filters1, padding=padding3)
    )

class HALNetResBlockIDSkip(nn.Module):
    def __init__(self, filters1, filters2,
                 padding_right1=1, padding_right2=0, padding_right3=0):
        super(HALNetResBlockIDSkip, self).__init__()
        self.right_res = HALNetResConvSequence(stride=1,
                                               filters1=filters1,
                                               filters2=filters2,
                                               padding1=padding_right1,
                                               padding2=padding_right2,
                                               padding3=padding_right3,
                                               first_in_channels=
                                               filters2)
        self.relu = nn.ReLU()

    def forward(self, input):
        left_res = input
        right_res = self.right_res(input)
        # element-wise sum
        out = left_res + right_res
        out = self.relu(out)
        return out

class HALNetResBlockConv(nn.Module):
    def __init__(self, stride, filters1, filters2, first_in_channels=0,
                 padding_left=0, padding_right1=0, padding_right2=0,
                 padding_right3=0):
        super(HALNetResBlockConv, self).__init__()
        if first_in_channels == 0:
            first_in_channels = filters1
        self.left_res = NetBlockConvBatchRelu(kernel_size=1, stride=stride,
                                              filters=filters2,
                                              padding=padding_left,
                                              in_channels=first_in_channels)
        self.right_res = HALNetResConvSequence(stride=stride,
                                               filters1=filters1,
                                               filters2=filters2,
                                               padding1=padding_right1,
                                               padding2=padding_right2,

                                               padding3=padding_right3,
                                               first_in_channels=
                                               first_in_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        left_res = self.left_res(input)
        right_res = self.right_res(input)
        # element-wise sum
        out = left_res + right_res
        out = self.relu(out)
        return out

def make_bilinear_weights(size, num_channels):
    ''' Make a 2D bilinear kernel suitable for upsampling
    Stack the bilinear kernel for application to tensor '''
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    filt = torch.from_numpy(filt)
    w = torch.zeros(num_channels, 1, size, size)
    for i in range(num_channels):
        w[i, 0] = filt
    return w

def parse_model_param(params_dict, key, default_value):
    try:
        ret = params_dict[key]
    except:
        if default_value == "Mandatory":
            # raise error again by trying to access value
            ret = params_dict[key]
        ret = default_value
    return ret

class HALNet(nn.Module):

    cross_entropy = False
    joint_ixs = None
    use_cuda = None
    WEIGHT_LOSS_INTERMED1 = 0.5
    WEIGHT_LOSS_INTERMED2 = 0.5
    WEIGHT_LOSS_INTERMED3 = 0.5
    WEIGHT_LOSS_MAIN = 1

    def __init__(self, params_dict):
        super(HALNet, self).__init__()
        # initialize variables
        self.joint_ixs = parse_model_param(params_dict, 'joint_ixs', default_value="Mandatory")
        self.use_cuda = parse_model_param(params_dict, 'use_cuda', default_value=False)
        self.num_joints = len(self.joint_ixs)
        self.cross_entropy = parse_model_param(params_dict, 'cross_entropy', default_value=False)
        # build network
        self.conv1 = cudafy(NetBlockConvBatchRelu(kernel_size=7, stride=1, filters=64,
                                                  in_channels=4, padding=3), self.use_cuda)
        self.mp1 = cudafy(nn.MaxPool2d(kernel_size=3, stride=2, padding=1), self.use_cuda)
        self.res2a = cudafy(HALNetResBlockConv(stride=1, filters1=64, filters2=256,
                                        padding_right1=1), self.use_cuda)
        self.res2b = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res2c = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res3a = cudafy(HALNetResBlockConv(stride=2, filters1=128, filters2=512,
                                               padding_right3=1, first_in_channels=256), self.use_cuda)
        self.interm_loss1 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=self.num_joints,
                                                         in_channels=512, padding=1), self.use_cuda)
        self.interm_loss1_deconv = cudafy(nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss1_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res3b = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res3c = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res4a = cudafy(HALNetResBlockConv(stride=2, filters1=256, filters2=1024,
                                        padding_right3=1,
                                        first_in_channels=512), self.use_cuda)
        self.interm_loss2 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=1024,
                                                         padding=1), self.use_cuda)
        self.interm_loss2_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss2_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res4b = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4c = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4d = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.conv4e = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=512,
                                                   in_channels=1024, padding=1), self.use_cuda)
        self.interm_loss3 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=512,
                                                         padding=1), self.use_cuda)
        self.interm_loss3_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss3_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.conv4f = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=256,
                                                   in_channels=512, padding=1), self.use_cuda)
        self.main_loss_conv = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                           filters=self.num_joints, in_channels=256,
                                                           padding=1), self.use_cuda)
        self.main_loss_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        if self.cross_entropy:
            self.softmax_final = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)

    def forward_common_net(self, x):
        out = self.conv1(x)
        out = self.mp1(out)
        out = self.res2a(out)
        out = self.res2b(out)
        out = self.res2c(out)
        res3aout = self.res3a(out)
        out = self.res3b(res3aout)
        out = self.res3c(out)
        res4aout = self.res4a(out)
        out = self.res4b(res4aout)
        out = self.res4c(out)
        out = self.res4d(out)
        conv4eout = self.conv4e(out)
        conv4fout = self.conv4f(conv4eout)
        return res3aout, res4aout, conv4eout, conv4fout

    def forward_subnet(self, x):
        res3aout, res4aout, conv4eout, conv4fout = self.forward_common_net(x)
        # intermediate losses
        # intermed 1
        out_intermed1 = self.interm_loss1(res3aout)
        out_intermed1 = self.interm_loss1_deconv(out_intermed1)
        if self.cross_entropy:
            out_intermed1 = self.interm_loss1_softmax(out_intermed1)
        # intermed 2
        out_intermed2 = self.interm_loss2(res4aout)
        out_intermed2 = self.interm_loss2_deconv(out_intermed2)
        if self.cross_entropy:
            out_intermed2 = self.interm_loss2_softmax(out_intermed2)
        # intermed 3
        out_intermed3 = self.interm_loss3(conv4eout)
        out_intermed3 = self.interm_loss3_deconv(out_intermed3)
        if self.cross_entropy:
            out_intermed3 = self.interm_loss3_softmax(out_intermed3)
        return out_intermed1, out_intermed2, out_intermed3, conv4fout,\
               res3aout, res4aout, conv4eout

    def forward_main_loss(self, conv4fout):
        out = self.main_loss_conv(conv4fout)
        out = self.main_loss_deconv(out)
        out_main = out
        # main loss
        if self.cross_entropy:
            out_main = self.softmax_final(out)
        return out_main

    def forward(self, x):
        # get subhalnet outputs (common to JORNet)
        out_intermed1, out_intermed2, out_intermed3, conv4fout, _, _, _ = self.forward_subnet(x)
        # out to main loss of halnet
        out_main = self.forward_main_loss(conv4fout)
        return out_intermed1, out_intermed2, out_intermed3, out_main


def _print_layer_output_shape(layer_name, output_shape):
    print("Layer " + layer_name + " output shape: " + str(output_shape))

def HALNetResConvSequence(stride, filters1, filters2,
                          padding1=1, padding2=0, padding3=0,
                          first_in_channels=0):
    if first_in_channels == 0:
        first_in_channels = filters1
    return nn.Sequential(
        # added padding = 1 to make shapes fit when joining
        # with left module
        NetBlockConvBatchRelu(kernel_size=1, stride=stride, filters=filters1,
                              in_channels=first_in_channels, padding=padding1),
        NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=filters1,
                              in_channels=filters1, padding=padding2),
        NetBlockConvBatchRelu(kernel_size=1, stride=1, filters=filters2,
                              in_channels=filters1, padding=padding3)
    )

class HALNetResBlockIDSkip(nn.Module):
    def __init__(self, filters1, filters2,
                 padding_right1=1, padding_right2=0, padding_right3=0):
        super(HALNetResBlockIDSkip, self).__init__()
        self.right_res = HALNetResConvSequence(stride=1,
                                               filters1=filters1,
                                               filters2=filters2,
                                               padding1=padding_right1,
                                               padding2=padding_right2,
                                               padding3=padding_right3,
                                               first_in_channels=
                                               filters2)
        self.relu = nn.ReLU()

    def forward(self, input):
        left_res = input
        right_res = self.right_res(input)
        # element-wise sum
        out = left_res + right_res
        out = self.relu(out)
        return out

class HALNetResBlockConv(nn.Module):
    def __init__(self, stride, filters1, filters2, first_in_channels=0,
                 padding_left=0, padding_right1=0, padding_right2=0,
                 padding_right3=0):
        super(HALNetResBlockConv, self).__init__()
        if first_in_channels == 0:
            first_in_channels = filters1
        self.left_res = NetBlockConvBatchRelu(kernel_size=1, stride=stride,
                                              filters=filters2,
                                              padding=padding_left,
                                              in_channels=first_in_channels)
        self.right_res = HALNetResConvSequence(stride=stride,
                                               filters1=filters1,
                                               filters2=filters2,
                                               padding1=padding_right1,
                                               padding2=padding_right2,

                                               padding3=padding_right3,
                                               first_in_channels=
                                               first_in_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        left_res = self.left_res(input)
        right_res = self.right_res(input)
        # element-wise sum
        out = left_res + right_res
        out = self.relu(out)
        return out

def make_bilinear_weights(size, num_channels):
    ''' Make a 2D bilinear kernel suitable for upsampling
    Stack the bilinear kernel for application to tensor '''
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    filt = torch.from_numpy(filt)
    w = torch.zeros(num_channels, 1, size, size)
    for i in range(num_channels):
        w[i, 0] = filt
    return w

def parse_model_param(params_dict, key, default_value):
    try:
        ret = params_dict[key]
    except:
        if default_value == "Mandatory":
            # raise error again by trying to access value
            ret = params_dict[key]
        ret = default_value
    return ret

class HALNet(nn.Module):

    cross_entropy = False
    joint_ixs = None
    use_cuda = None
    WEIGHT_LOSS_INTERMED1 = 0.5
    WEIGHT_LOSS_INTERMED2 = 0.5
    WEIGHT_LOSS_INTERMED3 = 0.5
    WEIGHT_LOSS_MAIN = 1

    def __init__(self, params_dict):
        super(HALNet, self).__init__()
        # initialize variables
        self.joint_ixs = parse_model_param(params_dict, 'joint_ixs', default_value="Mandatory")
        self.use_cuda = parse_model_param(params_dict, 'use_cuda', default_value=False)
        self.num_joints = len(self.joint_ixs)
        self.cross_entropy = parse_model_param(params_dict, 'cross_entropy', default_value=False)
        # build network
        self.conv1 = cudafy(NetBlockConvBatchRelu(kernel_size=7, stride=1, filters=64,
                                                  in_channels=4, padding=3), self.use_cuda)
        self.mp1 = cudafy(nn.MaxPool2d(kernel_size=3, stride=2, padding=1), self.use_cuda)
        self.res2a = cudafy(HALNetResBlockConv(stride=1, filters1=64, filters2=256,
                                        padding_right1=1), self.use_cuda)
        self.res2b = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res2c = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res3a = cudafy(HALNetResBlockConv(stride=2, filters1=128, filters2=512,
                                               padding_right3=1, first_in_channels=256), self.use_cuda)
        self.interm_loss1 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=self.num_joints,
                                                         in_channels=512, padding=1), self.use_cuda)
        self.interm_loss1_deconv = cudafy(nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss1_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res3b = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res3c = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res4a = cudafy(HALNetResBlockConv(stride=2, filters1=256, filters2=1024,
                                        padding_right3=1,
                                        first_in_channels=512), self.use_cuda)
        self.interm_loss2 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=1024,
                                                         padding=1), self.use_cuda)
        self.interm_loss2_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss2_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res4b = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4c = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4d = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.conv4e = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=512,
                                                   in_channels=1024, padding=1), self.use_cuda)
        self.interm_loss3 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=512,
                                                         padding=1), self.use_cuda)
        self.interm_loss3_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss3_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.conv4f = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=256,
                                                   in_channels=512, padding=1), self.use_cuda)
        self.main_loss_conv = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                           filters=self.num_joints, in_channels=256,
                                                           padding=1), self.use_cuda)
        self.main_loss_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        if self.cross_entropy:
            self.softmax_final = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)

    def forward_common_net(self, x):
        out = self.conv1(x)
        out = self.mp1(out)
        out = self.res2a(out)
        out = self.res2b(out)
        out = self.res2c(out)
        res3aout = self.res3a(out)
        out = self.res3b(res3aout)
        out = self.res3c(out)
        res4aout = self.res4a(out)
        out = self.res4b(res4aout)
        out = self.res4c(out)
        out = self.res4d(out)
        conv4eout = self.conv4e(out)
        conv4fout = self.conv4f(conv4eout)
        return res3aout, res4aout, conv4eout, conv4fout

    def forward_subnet(self, x):
        res3aout, res4aout, conv4eout, conv4fout = self.forward_common_net(x)
        # intermediate losses
        # intermed 1
        out_intermed1 = self.interm_loss1(res3aout)
        out_intermed1 = self.interm_loss1_deconv(out_intermed1)
        if self.cross_entropy:
            out_intermed1 = self.interm_loss1_softmax(out_intermed1)
        # intermed 2
        out_intermed2 = self.interm_loss2(res4aout)
        out_intermed2 = self.interm_loss2_deconv(out_intermed2)
        if self.cross_entropy:
            out_intermed2 = self.interm_loss2_softmax(out_intermed2)
        # intermed 3
        out_intermed3 = self.interm_loss3(conv4eout)
        out_intermed3 = self.interm_loss3_deconv(out_intermed3)
        if self.cross_entropy:
            out_intermed3 = self.interm_loss3_softmax(out_intermed3)
        return out_intermed1, out_intermed2, out_intermed3, conv4fout,\
               res3aout, res4aout, conv4eout

    def forward_main_loss(self, conv4fout):
        out = self.main_loss_conv(conv4fout)
        out = self.main_loss_deconv(out)
        out_main = out
        # main loss
        if self.cross_entropy:
            out_main = self.softmax_final(out)
        return out_main

    def forward(self, x):
        # get subhalnet outputs (common to JORNet)
        out_intermed1, out_intermed2, out_intermed3, conv4fout, _, _, _ = self.forward_subnet(x)
        # out to main loss of halnet
        out_main = self.forward_main_loss(conv4fout)
        return out_intermed1, out_intermed2, out_intermed3, out_main

In [0]:
class HALNet(nn.Module):

    cross_entropy = False
    joint_ixs = None
    use_cuda = None
    WEIGHT_LOSS_INTERMED1 = 0.5
    WEIGHT_LOSS_INTERMED2 = 0.5
    WEIGHT_LOSS_INTERMED3 = 0.5
    WEIGHT_LOSS_MAIN = 1

    def __init__(self):
        super(HALNet, self).__init__()
        # initialize variables
#         self.joint_ixs = parse_model_param(params_dict, 'joint_ixs', default_value="Mandatory")
        self.use_cuda = True #parse_model_param(params_dict, 'use_cuda', default_value=False)
        self.num_joints = 21#len(self.joint_ixs)
        self.cross_entropy =  nn.CrossEntropyLoss()# parse_model_param(params_dict, 'cross_entropy', default_value=False)
        # build network
        self.conv1 = cudafy(NetBlockConvBatchRelu(kernel_size=7, stride=1, filters=64,
                                                  in_channels=4, padding=3), self.use_cuda)
        self.mp1 = cudafy(nn.MaxPool2d(kernel_size=3, stride=2, padding=1), self.use_cuda)
        self.res2a = cudafy(HALNetResBlockConv(stride=1, filters1=64, filters2=256,
                                        padding_right1=1), self.use_cuda)
        self.res2b = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res2c = cudafy(HALNetResBlockIDSkip(filters1=64, filters2=256), self.use_cuda)
        self.res3a = cudafy(HALNetResBlockConv(stride=2, filters1=128, filters2=512,
                                               padding_right3=1, first_in_channels=256), self.use_cuda)
        self.interm_loss1 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=self.num_joints,
                                                         in_channels=512, padding=1), self.use_cuda)
        self.interm_loss1_deconv = cudafy(nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss1_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res3b = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res3c = cudafy(HALNetResBlockIDSkip(filters1=128, filters2=512), self.use_cuda)
        self.res4a = cudafy(HALNetResBlockConv(stride=2, filters1=256, filters2=1024,
                                        padding_right3=1,
                                        first_in_channels=512), self.use_cuda)
        self.interm_loss2 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=1024,
                                                         padding=1), self.use_cuda)
        self.interm_loss2_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss2_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.res4b = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4c = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.res4d = cudafy(HALNetResBlockIDSkip(filters1=256, filters2=1024), self.use_cuda)
        self.conv4e = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=512,
                                                   in_channels=1024, padding=1), self.use_cuda)
        self.interm_loss3 = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                         filters=self.num_joints, in_channels=512,
                                                         padding=1), self.use_cuda)
        self.interm_loss3_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        self.interm_loss3_softmax = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)
        self.conv4f = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1, filters=256,
                                                   in_channels=512, padding=1), self.use_cuda)
        self.main_loss_conv = cudafy(NetBlockConvBatchRelu(kernel_size=3, stride=1,
                                                           filters=self.num_joints, in_channels=256,
                                                           padding=1), self.use_cuda)
        self.main_loss_deconv = cudafy(nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True), self.use_cuda)
        if self.cross_entropy:
            self.softmax_final = cudafy(NetBlockSoftmaxLogProbability2D(), self.use_cuda)

    def forward_common_net(self, x):
        out = self.conv1(x)
        out = self.mp1(out)
        out = self.res2a(out)
        out = self.res2b(out)
        out = self.res2c(out)
        res3aout = self.res3a(out)
        out = self.res3b(res3aout)
        out = self.res3c(out)
        res4aout = self.res4a(out)
        out = self.res4b(res4aout)
        out = self.res4c(out)
        out = self.res4d(out)
        conv4eout = self.conv4e(out)
        conv4fout = self.conv4f(conv4eout)
        return res3aout, res4aout, conv4eout, conv4fout

    def forward_subnet(self, x):
        res3aout, res4aout, conv4eout, conv4fout = self.forward_common_net(x)
        # intermediate losses
        # intermed 1
        out_intermed1 = self.interm_loss1(res3aout)
        out_intermed1 = self.interm_loss1_deconv(out_intermed1)
        if self.cross_entropy:
            out_intermed1 = self.interm_loss1_softmax(out_intermed1)
        # intermed 2
        out_intermed2 = self.interm_loss2(res4aout)
        out_intermed2 = self.interm_loss2_deconv(out_intermed2)
        if self.cross_entropy:
            out_intermed2 = self.interm_loss2_softmax(out_intermed2)
        # intermed 3
        out_intermed3 = self.interm_loss3(conv4eout)
        out_intermed3 = self.interm_loss3_deconv(out_intermed3)
        if self.cross_entropy:
            out_intermed3 = self.interm_loss3_softmax(out_intermed3)
        return out_intermed1, out_intermed2, out_intermed3, conv4fout,\
               res3aout, res4aout, conv4eout

    def forward_main_loss(self, conv4fout):
        out = self.main_loss_conv(conv4fout)
        out = self.main_loss_deconv(out)
        out_main = out
        # main loss
        if self.cross_entropy:
            out_main = self.softmax_final(out)
        return out_main

    def forward(self, x):
        # get subhalnet outputs (common to JORNet)
        out_intermed1, out_intermed2, out_intermed3, conv4fout, _, _, _ = self.forward_subnet(x)
        # out to main loss of halnet
        out_main = self.forward_main_loss(conv4fout)
        return out_intermed1, out_intermed2, out_intermed3, out_main

In [0]:
def dims_fun(w, h):
  
    w, h = (w-7+2*3)/1+1, (h-7+2*3)/1+1 #conv1
    w, h = (w-3+2*1)/2+1, (h-3+2*1)/2+1 #maxpool
    w, h = (w-1+2*0)/1+1, (h-1+2*0)/1+1 #res2out
    w, h = (w-1+2*0)/2+1, (h-1+2*0)/2+1 #res3out
    w,h = int(w),int(h)
    res3out = w * h * 512
   
    w, h = (w-1+2*0)/2+1, (h-1+2*0)/2+1 #res4out
    w,h = int(w),int(h)
    res4out = w * h * 1024
   
    w, h = (w-3+2*1)/1+1, (h-3+2*1)/1+1 #conv4e
    w,h = int(w),int(h)
    conv4e = w*h*512

    w, h = (w-3+2*1)/1+1, (h-3+2*1)/1+1 #conv4f
    w,h = int(w),int(h)
    conv4f = w*h*256
    return res3out,res4out,conv4e,conv4f,h,w


In [0]:

class JORNet(HALNet):

    #innerprod1_size = 65536

    def map_out_to_loss(self, innerprod1_size):
        return cudafy(nn.Linear(in_features=innerprod1_size, out_features=200), self.use_cuda)

    def map_out_conv(self, in_channels):
        return cudafy(HALNet.HALNetConvBlock(
            kernel_size=3, stride=1, filters=21, in_channels=in_channels, padding=1),
            self.use_cuda)

    def __init__(self,dims,num_joints=20):
        super(JORNet, self).__init__()
        
        res3out,res4out,conv4e,conv4f,h,w = dims_fun(dims[0],dims[1])
#         self.innerprod1_size = 256 * h * w
        self.crop_res = dims
        self.num_joints = num_joints
#         self.main_loss_conv = cudafy(HALNet.HALNetConvBlock(
#                 kernel_size=3, stride=1, filters=21, in_channels=256, padding=1),
#             self.use_cuda)
        self.main_loss_deconv1 = cudafy(nn.Upsample(size=self.crop_res, mode='bilinear'), self.use_cuda)
        #self.main_loss_deconv2 = cudafy(nn.Upsample(scale_factor=1, mode='bilinear'),
        #                                self.use_cuda)
        if self.cross_entropy:
            self.softmax_final = cudafy(HALNet.
                                        SoftmaxLogProbability2D(), self.use_cuda)
        self.innerproduct1_joint1 = cudafy(
            nn.Linear(in_features=res3out, out_features=200), self.use_cuda)
        self.innerproduct2_joint1 = cudafy(
            nn.Linear(in_features=200, out_features=self.num_joints * 3), self.use_cuda)

        self.innerproduct1_joint2 = cudafy(
            nn.Linear(in_features=res4out, out_features=200), self.use_cuda)
        self.innerproduct2_joint2 = cudafy(
            nn.Linear(in_features=200, out_features=self.num_joints * 3), self.use_cuda)

        self.innerproduct1_joint3 = cudafy(
            nn.Linear(in_features=conv4e, out_features=200), self.use_cuda)
        self.innerproduct2_joint3 = cudafy(
            nn.Linear(in_features=200, out_features=self.num_joints * 3), self.use_cuda)

        self.innerproduct1_joint_main = cudafy(
            nn.Linear(in_features=conv4f, out_features=200), self.use_cuda)
        self.innerproduct2_join_main = cudafy(
            nn.Linear(in_features=200, out_features=self.num_joints * 3), self.use_cuda)

    def forward(self, x):
        out_intermed_hm1, out_intermed_hm2, out_intermed_hm3, conv4fout, \
        res3aout, res4aout, conv4eout = self.forward_subnet(x)
        out_intermed_hm_main = self.forward_main_loss(conv4fout)
        innerprod1_size = res3aout.shape[1] * res3aout.shape[2] * res3aout.shape[3]
        out_intermed_j1 = res3aout.view(-1, innerprod1_size)
        out_intermed_j1 = self.innerproduct1_joint1(out_intermed_j1)
        out_intermed_j1 = self.innerproduct2_joint1(out_intermed_j1)

        innerprod1_size = res4aout.shape[1] * res4aout.shape[2] * res4aout.shape[3]
        out_intermed_j2 = res4aout.view(-1, innerprod1_size)
        out_intermed_j2 = self.innerproduct1_joint2(out_intermed_j2)
        out_intermed_j2 = self.innerproduct2_joint2(out_intermed_j2)

        innerprod1_size = conv4eout.shape[1] * conv4eout.shape[2] * conv4eout.shape[3]
        out_intermed_j3 = conv4eout.view(-1, innerprod1_size)
        out_intermed_j3 = self.innerproduct1_joint3(out_intermed_j3)
        out_intermed_j3 = self.innerproduct2_joint3(out_intermed_j3)

        innerprod1_size = conv4fout.shape[1] * conv4fout.shape[2] * conv4fout.shape[3]
        out_intermed_j_main = conv4fout.view(-1, innerprod1_size)
        out_intermed_j_main = self.innerproduct1_joint_main(out_intermed_j_main)
        out_intermed_j_main = self.innerproduct2_join_main(out_intermed_j_main)

        return out_intermed_hm1, out_intermed_hm2, out_intermed_hm3, out_intermed_hm_main,\
               out_intermed_j1, out_intermed_j2, out_intermed_j3, out_intermed_j_main

In [0]:
# halnet = HALNet()

In [0]:
# a = halnet.forward_common_net(rgbd_img[None])

In [0]:
dims = (128,250)
rgbd_img = torch.rand((4,dims[0],dims[1]))
rgbd_img = rgbd_img.cuda()

In [0]:
jornet = JORNet(dims,num_joints=21)

In [0]:
# jornet(rgbd_img[None])


In [0]:
from time import time
i = 0
with torch.no_grad():
  
    t = time()
    while True:
        i+=1
        jornet(rgbd_img[None])
        
        if time()-t>1:      
            print("time: {} s".format(time()-t))
            print("number of feedforward: {}".format(i))
            break

time: 1.0158119201660156 s
number of feedforward: 43
